In [1]:
import sys
sys.path.insert(0, '/das/work/p17/p17983/')

In [2]:
%matplotlib nbagg
import numpy as np
import json
import os
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

from alvra_tools.load_data import *
from alvra_tools.channels import *
from alvra_tools.utils import errfunc_sigma, errfunc_fwhm

[INFO][corrections] Numba not available, reverting to Numpy


Set up the filters.  "numstds" is the number of standard deviations to take from the median.

Plot some diagnostics

In [3]:
def get_XAS(scan_name, DIR, DIR_json, numstds, minIzero):
    

    json_file = DIR_json + scan_name + "_scan_info.json"
    print (json_file)

    with open(json_file) as file:
        data = json.load(file)
    numFiles = len(data['scan_files'])

    print ("Processing",numFiles,"files")

#    DataFluo_pump_norm_total = np.empty(0)
#    DataFluo_unpump_norm_total = np.empty(0)
#    err_DataFluo_pump_total = np.empty(0)
#    err_DataFluo_unpump_total = np.empty(0)

#    IzeroFEL_pump_original_total = np.empty(0)
#    IzeroFEL_pump_total = np.empty(0)
#    IzeroFEL_unpump_original_total = np.empty(0)
#    IzeroFEL_unpump_total = np.empty(0)

#    DataFluo_pump_original_total = np.empty(0)
#    DataFluo_pump_total = np.empty(0)
#    DataFluo_unpump_original_total = np.empty(0)
#    DataFluo_unpump_total = np.empty(0)

#    Energy_eV = np.empty(0)
#    iZero = np.empty(0)

    for i in range(0,numFiles):
    #for i in range(0,1):
        filename = str(data['scan_files'][i][0])
        filename = DIR + os.path.basename(filename)
        exists = os.path.isfile(filename)
        if not exists:
            print("No such file")
        elif exists: #Do we need this 39?
#             print("step",i+1,"of",numFiles,": Processing %s" %(str(data['scan_files'][i][0])))
            (DataFluo_pump, DataFluo_unpump, IzeroFEL_pump, IzeroFEL_unpump, Energy, _, _, _) = \
                load_PumpProbe_events(filename, channel_energy)

            IzeroFEL_pump_original = IzeroFEL_pump.copy()
            IzeroFEL_unpump_original = IzeroFEL_unpump.copy()

            IzeroFEL_pump_original_total = np.append(IzeroFEL_pump_original_total, IzeroFEL_pump_original)
            IzeroFEL_unpump_original_total = np.append(IzeroFEL_unpump_original_total, IzeroFEL_unpump_original)

            DataFluo_pump_original_total = np.append(DataFluo_pump_original_total, DataFluo_pump)
            DataFluo_unpump_original_total = np.append(DataFluo_unpump_original_total, DataFluo_unpump)

            IzeroMedian = np.median(np.concatenate([IzeroFEL_pump, IzeroFEL_unpump]))
            IzeroSTD = np.std(np.concatenate([IzeroFEL_pump, IzeroFEL_unpump]))

            conditionPumpMax = IzeroFEL_pump < IzeroMedian+numstds*IzeroSTD
            conditionPumpMin = IzeroFEL_pump > IzeroMedian-numstds*IzeroSTD
            conditionPumpLow = IzeroFEL_pump > minIzero

            conditionUnPumpMax = IzeroFEL_unpump < IzeroMedian+numstds*IzeroSTD
            conditionUnPumpMin = IzeroFEL_unpump > IzeroMedian-numstds*IzeroSTD
            conditionUnPumpLow = IzeroFEL_unpump > minIzero

            condIzeroPump = conditionPumpMax & conditionPumpMin & conditionPumpLow
            condIzeroUnPump = conditionUnPumpMax & conditionUnPumpMin & conditionUnPumpLow

            IzeroFEL_pump = IzeroFEL_pump[condIzeroPump]
            IzeroFEL_unpump = IzeroFEL_unpump[condIzeroUnPump]

            IzeroFEL_pump_total = np.append(IzeroFEL_pump_total, IzeroFEL_pump)
            IzeroFEL_unpump_total = np.append(IzeroFEL_unpump_total, IzeroFEL_unpump)

            DataFluo_pump = DataFluo_pump[condIzeroPump]
            DataFluo_unpump = DataFluo_unpump[condIzeroUnPump]

#             print(DataFluo_pump.shape)

#            DataFluo_pump_total = np.append(DataFluo_pump_total, DataFluo_pump)
#            DataFluo_unpump_total = np.append(DataFluo_unpump_total, DataFluo_unpump)

#            DataFluo_pump_norm = DataFluo_pump/IzeroFEL_pump
#            DataFluo_unpump_norm = DataFluo_unpump/IzeroFEL_unpump

#            DataFluo_pump_norm_total = np.append(DataFluo_pump_norm_total, DataFluo_pump_norm.mean())
#            DataFluo_unpump_norm_total = np.append(DataFluo_unpump_norm_total, DataFluo_unpump_norm.mean())
#            err_DataFluo_pump_total = np.append(err_DataFluo_pump_total, DataFluo_pump_norm.std()/np.sqrt(DataFluo_pump_norm.size))
#            err_DataFluo_unpump_total = np.append(err_DataFluo_unpump_total, DataFluo_unpump_norm.std()/np.sqrt(DataFluo_unpump_norm.size))

#            Energy = [x for x in Energy if (np.abs(x) > 0)]
#            Energy_eV = np.append(Energy_eV, np.mean(Energy))
#            iZero = [x for x in iZero if (np.abs(x) > 0)]
#            iZero = np.append(iZero, np.mean(IzeroFEL_pump_total))
            
#     print("The original number of pumped and unpumped shots is:")
#     print(IzeroFEL_pump_original_total.shape, IzeroFEL_unpump_original_total.shape)
#     print("The filtered number of pumped and unpumped shots is:")
#     print(IzeroFEL_pump_total.shape, IzeroFEL_unpump_total.shape)

    
#     print(Energy_eV.shape)
#     print(iZero.shape)
#     print(DataFluo_pump_norm_total.shape)
#     print(DataFluo_unpump_norm_total.shape)


    



#    return iZero, Energy_eV, DataFluo_unpump_norm_total, DataFluo_pump_norm_total
    return IzeroFEL_pump, IzeroFEL_unpump, IzeroFEL_pump_total, IzeroFEL_unpump_total, DataFluo_pump, DataFluo_unpump, Energy
        

In [4]:
ploton = True

basename = 'RuBpy3_monoscan_'
scanlist = [7, 8]


numstds = 2
minIzero = 0.015

for i in range(0, len(scanlist)):
    if scanlist[i] < 10:
        scan_name = basename + '00' + str(scanlist[i])
    else:
        scan_name = basename + '0' + str(scanlist[i])
        
    
    DIR = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name + "/"
    DIR_json = "/sf/alvra/data/p17983/res/scan_info/"
    
    Fluo_p_all = np.empty(0)
    Fluo_u_all = np.empty(0)
    err_Fluo_p_all = np.empty(0)
    err_Fluo_u_all = np.empty(0)

    IzeroFEL_p_original_all = np.empty(0)
    IzeroFEL_p_all = np.empty(0)
    IzeroFEL_u_original_all = np.empty(0)
    IzeroFEL_u_all = np.empty(0)

    Fluo_p_original_all = np.empty(0)
    Fluo_p_all = np.empty(0)
    Fluo_u_original_all = np.empty(0)
    Fluo_u_all = np.empty(0)

    Energy_eV = np.empty(0)
    iZero = np.empty(0)
    
    (IzeroFEL_pump, IzeroFEL_unpump, IzeroFEL_pump_total, IzeroFEL_unpump_total, Fluo_p, Fluo_u, Energy) = get_XAS(scan_name, DIR, DIR_json, numstds, minIzero)
    
  
    
    Fluo_p_all = np.append(Fluo_p_all, Fluo_p)
    Fluo_u_all = np.append(Fluo_u_all, Fluo_u)

    Fluo_p_norm = Fluo_p/IzeroFEL_pump
    Fluo_u_norm = Fluo_u/IzeroFEL_unpump

    Fluo_p_norm_all = np.append(Fluo_p_norm_all, Fluo_p_norm.mean())
    Fluo_u_norm_all = np.append(Fluo_u_norm_all, Fluo_u_norm.mean())
    err_Fluo_p_all = np.append(err_Fluo_p_total, Fluo_p_norm.std()/np.sqrt(Fluo_p_norm.size))
    err_Fluo_u_all = np.append(err_Fluo_u_total, Fluo_u_norm.std()/np.sqrt(Fluo_u_norm.size))

    Energy = [x for x in Energy if (np.abs(x) > 0)]
    Energy_eV = np.append(Energy_eV, np.mean(Energy))
    iZero = [x for x in iZero if (np.abs(x) > 0)]
    iZero = np.append(iZero, np.mean(IzeroFEL_pump_total))
            
    Energy_eV = Energy_eV[1:]
    Fluo_p_norm_all = Fluo_p_norm_all[1:]
    Fluo_u_norm_all = Fluo_u_norm_all[1:]
    iZero =iZero[1:]
            
    plt.figure()
    _, bins, _ = plt.hist(IzeroFEL_p_original_all, 100, label = 'unfiltered')
    _ = plt.hist(IzeroFEL_p_all, bins, rwidth = .5, label = 'filtered')
    plt.title('Izero, pumped')
    plt.legend()

    plt.figure()
    _, bins, _ = plt.hist(IzeroFEL_u_original_all, 100, label = 'unfiltered')
    _ = plt.hist(IzeroFEL_u_all, bins, rwidth = .5, label = 'filtered')
    plt.title('Izero, unpumped')
    plt.legend()
    
    plt.figure()
    plt.scatter(IzeroFEL_p_original_all, Fluo_p_original_all)
    plt.scatter(IzeroFEL_p_all, Fluo_p_all)
    plt.title('Izero, pumped')
    plt.xlabel('I0')
    plt.ylabel('Absorption')  
    
    
    plt.figure()
    plt.plot(np.array(Energy_eV), Fluo_p_norm_all,label='Pumped')
    plt.plot(np.array(Energy_eV), Fluo_u_norm_all,label='UnPumped')
    plt.plot(np.array(Energy_eV), np.array(iZero)*100,label="I_zero_Arb")
    plt.xlabel('energy (eV)')
    plt.ylabel('absorption')
    plt.title('XAS_'+ scan_name)
    plt.legend()
        

/sf/alvra/data/p17983/res/scan_info/RuBpy3_monoscan_007_scan_info.json
Processing 36 files


UnboundLocalError: local variable 'IzeroFEL_pump_original_total' referenced before assignment

In [ ]:
ploton = True


numstds = 2
minIzero = 0.015

scan_name_01 = "RuBpy3_monoscan_009"
# scan_name = "Ru_foil_monoscan_003"

# sf/alvra/data/p17983/raw/scan_data/Ru_foil_monoscan_001/
DIR = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name_01 + "/"
# DIR_json = "/das/work/p17/p17983/Example_data/res/scan_info/"
DIR_json = "/sf/alvra/data/p17983/res/scan_info/"

(iZero_01,Energy_eV_01,DataFluo_unpump_norm_total_01, DataFluo_pump_norm_total_01) = get_XAS(scan_name_01, DIR, DIR_json, numstds, minIzero)

# scan_name = "Ru_foil_monoscan_003"
scan_name_02 = "RuBpy3_monoscan_008"

DIR = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name_02 + "/"
# DIR_json = "/das/work/p17/p17983/Example_data/res/scan_info/"
DIR_json = "/sf/alvra/data/p17983/res/scan_info/"
(iZero_02,Energy_eV_02, DataFluo_unpump_norm_total_02, DataFluo_pump_norm_total_02) = get_XAS(scan_name_02, DIR, DIR_json, numstds, minIzero)
    
scan_name_03 = "RuBpy3_monoscan_007"
# scan_name = "Ru_foil_monoscan_003"

# sf/alvra/data/p17983/raw/scan_data/Ru_foil_monoscan_001/
DIR = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name_03 + "/"
# DIR_json = "/das/work/p17/p17983/Example_data/res/scan_info/"
DIR_json = "/sf/alvra/data/p17983/res/scan_info/"

(iZero_03,Energy_eV_03,DataFluo_unpump_norm_total_03, DataFluo_pump_norm_total_03) = get_XAS(scan_name_03, DIR, DIR_json, numstds, minIzero)
    
        

In [ ]:
#get rid of zero offset before comparing spectra

offset_min = 0.1
DataFluo_pump_norm_total_01 = DataFluo_pump_norm_total_01-min(DataFluo_unpump_norm_total_01)+offset_min
DataFluo_unpump_norm_total_01 = DataFluo_unpump_norm_total_01-min(DataFluo_unpump_norm_total_01)+offset_min

DataFluo_pump_norm_total_02 = DataFluo_pump_norm_total_02-min(DataFluo_unpump_norm_total_02)+offset_min
DataFluo_unpump_norm_total_02 = DataFluo_unpump_norm_total_02-min(DataFluo_unpump_norm_total_02)+offset_min

DataFluo_pump_norm_total_03 = DataFluo_pump_norm_total_03-min(DataFluo_unpump_norm_total_03)+offset_min
DataFluo_unpump_norm_total_03 = DataFluo_unpump_norm_total_03-min(DataFluo_unpump_norm_total_03)+offset_min

In [ ]:
scaling = 1

plt.figure()
plt.plot(np.array(Energy_eV_01), DataFluo_pump_norm_total_01, label='Pumped'+scan_name_01)
#plt.plot(np.array(Energy_eV_01), DataFluo_unpump_norm_total_01, label='UnPumped'+scan_name_01)
plt.plot(np.array(Energy_eV_02), DataFluo_pump_norm_total_02*scaling, label='Pumped'+scan_name_02) 
#plt.plot(np.array(Energy_eV_02), DataFluo_unpump_norm_total_02*scaling, label='UnPumped'+scan_name_02)
#plt.plot(np.array(Energy_eV_03), DataFluo_pump_norm_total_03*scaling, label='Pumped'+scan_name_03) 
#plt.plot(np.array(Energy_eV_03), DataFluo_unpump_norm_total_03*scaling, label='UnPumped'+scan_name_03)
plt.legend()
plt.title('XAS comparison ')
plt.ylabel('totally arb')
plt.xlabel('energy(ev)')

In [ ]:
differential = (DataFluo_pump_norm_total_01-DataFluo_unpump_norm_total_01)#/DataFluo_unpump_norm_total_01
plt.figure()
plt.plot(np.array(Energy_eV_01), differential, label='not normalized')
plt.plot(np.array(Energy_eV_01), differential/DataFluo_unpump_norm_total_01, label='normalized')
plt.xlabel('energy(ev)')
plt.title('Differential Spectrum '+scan_name_01)
plt.ylabel('delta Alpha')
plt.legend()

# plt.xlim([2837.5,2850])
# plt.ylim([-.5,.5])

In [ ]:
differential = (DataFluo_pump_norm_total_02-DataFluo_unpump_norm_total_02)
plt.figure()
plt.plot(np.array(Energy_eV_02), differential, label='not normalized')
plt.plot(np.array(Energy_eV_02), differential/DataFluo_unpump_norm_total_02, label='normalized')
plt.xlabel('energy(ev)')
plt.title('Differential Spectrum '+scan_name_02)
plt.ylabel('delta Alpha')
plt.legend()

# plt.ylim([-.5,.5])


d1 = (DataFluo_pump_norm_total_01-DataFluo_unpump_norm_total_01)/DataFluo_unpump_norm_total_01
d2 = (DataFluo_pump_norm_total_02-DataFluo_unpump_norm_total_02)/DataFluo_unpump_norm_total_02
d3 = (DataFluo_pump_norm_total_03-DataFluo_unpump_norm_total_03)/DataFluo_unpump_norm_total_03

d23 = 0.5*(d2 + d3)

plt.figure()
plt.plot(np.array(Energy_eV_01), d1, label=scan_name_01)
plt.plot(np.array(Energy_eV_02), d23, label='ave ' + scan_name_02 + ', ' + scan_name_03)
plt.xlabel('energy(ev)')
plt.title('Differential Spectra')
plt.ylabel('delta Alpha')
plt.legend()